# Check parallel envs 

In [ ]:
import torch
import matplotlib.pyplot as plt
h=40
w=40
position_ids = None
if position_ids is None:
    radius = h
    norm_dist = torch.distributions.normal.Normal(0, radius // 4)
    x = torch.linspace(-radius, radius, radius + 1)
    kern1d = norm_dist.cdf(x)
    kern1d = kern1d[1:] - kern1d[:-1]
    kern2d = torch.ger(kern1d, kern1d)
    position_ids = torch.round(radius**2 * kern2d)
    position_ids = position_ids.unsqueeze(0).reshape(1, h, w)
    

plt.matshow(position_ids.squeeze(0).numpy())

In [ ]:
import json
import gzip
with gzip.open('/data/zirshad/VLNCE-data/data/datasets/R2R_VLNCE_v1_preprocessed/val_seen/val_seen.json.gz') as f:
    data = json.load(f)
    
num_to_actions={0:'Stop',
                1:'Move Forward',
                2:'Turn Left',
                3:'Turn Right',
                4:'Look Up',
                5:'Look Down'}

# print(data['instruction_vocab']['stoi']
# )

episode = 0


ep_id=164
print(data['episodes'][ep_id]
)

import numpy as np
myFile = np.genfromtxt('/home/zirshad/SASRA/videos/debug/seq2seq_text_attn/run_7/episode=165-ckpt=0-SPL=1.00attention.csv', delimiter=',')
actions = np.genfromtxt('/home/zirshad/SASRA/videos/debug/seq2seq_text_attn/run_7/episode=165-ckpt=0-SPL=1.00action.csv', delimiter=',')

actions_name =[]
print(myFile[0].shape)
print(myFile[0])
print(actions.shape)

for i in range(len(actions)):
    actions_name.append(num_to_actions[actions[i]])
    
instruction = "CLS "+ data['episodes'][ep_id]['instruction']['instruction_text'] + "EOS"

print(actions_name)
print(actions)
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot()
cax = ax.matshow(myFile, cmap='bone')
fig.colorbar(cax)
fig.set_size_inches(20, 20)
plt.xticks(np.arange(0, myFile.shape[1], 1.0))
plt.yticks(np.arange(0, myFile.shape[0], 1.0))
ax.set_xticklabels(instruction.split(" "), rotation=90)
ax.set_yticklabels(actions_name)






In [1]:
from vlnce_baselines.common.env_utils import construct_envs
from habitat import Config
from vlnce_baselines.config.default import get_config
from habitat_baselines.common.environments import get_env_class

#Import config and perform some manipulation
exp_config ='vlnce_baselines/config/paper_configs/seq2seq_sem_attn.yaml'
config = get_config(exp_config, None)
split = config.TASK_CONFIG.DATASET.SPLIT
config.defrost()
config.TASK_CONFIG.TASK.NDTW.SPLIT = split
config.TASK_CONFIG.TASK.SDTW.SPLIT = split

# if doing teacher forcing, don't switch the scene until it is complete
if config.DAGGER.P == 1.0:
    config.TASK_CONFIG.ENVIRONMENT.ITERATOR_OPTIONS.MAX_SCENE_REPEAT_STEPS = (
        -1
    )
config.freeze()
envs = construct_envs(config, get_env_class(config.ENV_NAME))
obs=envs.reset()

2020-08-05 21:30:05,661 Initializing dataset VLN-CE-v1
2020-08-05 21:30:06,591 [construct_envs] Using GPU ID 1


In [2]:
import torch
from vlnce_baselines.common.utils import transform_obs
from habitat_baselines.common.utils import batch_obs

device = (
    torch.device("cuda", config.TORCH_GPU_ID)
    if torch.cuda.is_available()
    else torch.device("cpu")
)
print(envs.num_envs)
print(envs.current_episodes()[0].episode_id)
observations = envs.reset()
# observations[0]['instruction_text'] = 'Go to kitchen'

for k,v in observations[0].items():
    print(k)
# print(observations[0])

observations = transform_obs(
    observations, config.TASK_CONFIG.TASK.INSTRUCTION_SENSOR_UUID, is_bert=True
)

# # print(observations)
batch = batch_obs(observations, device=device)

# print(batch['instruction_batch'])


# print (batch["instruction_batch"].shape)
# lengths = (batch['instruction_batch'] != 0.0).long().sum(dim=1)
# print(lengths)



1
8979
rgb
depth
semantic
instruction
vln_oracle_action_sensor
progress
heading
ego_sem_map


In [3]:
print(batch['instruction'].shape)

torch.Size([1, 200])


In [ ]:
envs.close()

# Inspect Dataset, pytorch Dataloader, padding and lmdb load

In [ ]:
#Import config and perform some manipulation
from vlnce_baselines.config.default import get_config
exp_config ='vlnce_baselines/config/paper_configs/seq2seq_sem_attn.yaml'
config = get_config(exp_config, None)
split = config.TASK_CONFIG.DATASET.SPLIT
config.defrost()
config.TASK_CONFIG.TASK.NDTW.SPLIT = split
config.TASK_CONFIG.TASK.SDTW.SPLIT = split

# if doing teacher forcing, don't switch the scene until it is complete
if config.DAGGER.P == 1.0:
    config.TASK_CONFIG.ENVIRONMENT.ITERATOR_OPTIONS.MAX_SCENE_REPEAT_STEPS = (
        -1
    )
config.freeze()

import torch
import copy
import gc
import json
import os
import random
import time
import warnings
from collections import defaultdict
from typing import Dict

import lmdb
import msgpack_numpy
import numpy as np
import torch
import torch.nn.functional as F
import tqdm

class ObservationsDict(dict):
    def pin_memory(self):
        for k, v in self.items():
            self[k] = v.pin_memory()

        return self


def collate_fn(batch):
    """Each sample in batch: (
            obs,
            prev_actions,
            oracle_actions,
            inflec_weight,
        )
    """

    def _pad_helper(t, max_len, fill_val=0):
        pad_amount = max_len - t.size(0)
        if pad_amount == 0:
            return t
        
        pad = torch.full_like(t[0:1], fill_val).expand(pad_amount, *t.size()[1:])
        return torch.cat([t, pad], dim=0)

    transposed = list(zip(*batch))

    observations_batch = list(transposed[0])
    prev_actions_batch = list(transposed[1])
    corrected_actions_batch = list(transposed[2])
    N = len(corrected_actions_batch)
    weights_batch = list(transposed[3])
    B = len(prev_actions_batch)
    new_observations_batch = defaultdict(list)
    for sensor in observations_batch[0]:
        if sensor == 'instruction':
            for bid in range(N):
                new_observations_batch[sensor].append(observations_batch[bid][sensor])
        else: 
            for bid in range(B):
                new_observations_batch[sensor].append(observations_batch[bid][sensor])

    observations_batch = new_observations_batch

    max_traj_len = max(ele.size(0) for ele in prev_actions_batch)
#     obs_lengths=[]
    for bid in range(B):
        for sensor in observations_batch:
            if sensor == 'instruction':
                continue
            observations_batch[sensor][bid] = _pad_helper(
                observations_batch[sensor][bid], max_traj_len, fill_val=0.0
            )
#         obs_lengths.append(prev_actions_batch[bid].shape[0])
        prev_actions_batch[bid] = _pad_helper(prev_actions_batch[bid], max_traj_len)
        corrected_actions_batch[bid] = _pad_helper(
            corrected_actions_batch[bid], max_traj_len, fill_val=-1.0
        )
        weights_batch[bid] = _pad_helper(weights_batch[bid], max_traj_len)

    for sensor in observations_batch:
        observations_batch[sensor] = torch.stack(observations_batch[sensor], dim=1)
        observations_batch[sensor] = observations_batch[sensor].transpose(1,0)
        observations_batch[sensor] = observations_batch[sensor].contiguous().view(
            -1, *observations_batch[sensor].size()[2:]
        )
        # if sensor == 'instruction':
        #     observations_batch[sensor] = observations_batch[sensor].contiguous().view(
        #    -1, *observations_batch[sensor].size()[2:]
        #  )


    prev_actions_batch = torch.stack(prev_actions_batch, dim=1)
    corrected_actions_batch = torch.stack(corrected_actions_batch, dim=1)
    weights_batch = torch.stack(weights_batch, dim=1)
    not_done_masks = torch.ones_like(corrected_actions_batch, dtype=torch.float)
    not_done_masks[0] = 0
    
    prev_actions_batch = prev_actions_batch.transpose(1,0)
    not_done_masks = not_done_masks.transpose(1,0)
    corrected_actions_batch = corrected_actions_batch.transpose(1,0)
    weights_batch = weights_batch.transpose(1,0)
    
    observations_batch = ObservationsDict(observations_batch)

    return (
        observations_batch,
        prev_actions_batch.contiguous().view(-1, 1),
        not_done_masks.contiguous().view(-1, 1),
        corrected_actions_batch,
        weights_batch,
    )
    # return (
    #     observations_batch,
    #     prev_actions_batch,
    #     not_done_masks,
    #     corrected_actions_batch,
    #     weights_batch,
    # )


def _block_shuffle(lst, block_size):
    blocks = [lst[i : i + block_size] for i in range(0, len(lst), block_size)]
    random.shuffle(blocks)

    return [ele for block in blocks for ele in block]


class IWTrajectoryDataset(torch.utils.data.IterableDataset):
    def __init__(
        self,
        lmdb_features_dir,
        use_iw,
        inflection_weight_coef=1.0,
        lmdb_map_size=1e9,
        batch_size=1,
    ):
        super().__init__()
        self.lmdb_features_dir = lmdb_features_dir
        self.lmdb_map_size = lmdb_map_size
        self.preload_size = batch_size * 100
        self._preload = []
        self.batch_size = batch_size

        if use_iw:
            self.inflec_weights = torch.tensor([1.0, inflection_weight_coef])
        else:
            self.inflec_weights = torch.tensor([1.0, 1.0])

        with lmdb.open(
            self.lmdb_features_dir,
            map_size=int(self.lmdb_map_size),
            readonly=True,
            lock=False,
        ) as lmdb_env:
            self.length = lmdb_env.stat()["entries"]

    def _load_next(self):
        if len(self._preload) == 0:
            if len(self.load_ordering) == 0:
                raise StopIteration

            new_preload = []
            lengths = []
            with lmdb.open(
                self.lmdb_features_dir,
                map_size=int(self.lmdb_map_size),
                readonly=True,
                lock=False,
            ) as lmdb_env, lmdb_env.begin(buffers=True) as txn:
                for _ in range(self.preload_size):
                    if len(self.load_ordering) == 0:
                        break

                    new_preload.append(
                        msgpack_numpy.unpackb(
                            txn.get(str(self.load_ordering.pop()).encode()), raw=False
                        )
                    )

                    lengths.append(len(new_preload[-1][0]))

            sort_priority = list(range(len(lengths)))
            random.shuffle(sort_priority)

            sorted_ordering = list(range(len(lengths)))
            sorted_ordering.sort(key=lambda k: (lengths[k], sort_priority[k]))

            for idx in _block_shuffle(sorted_ordering, self.batch_size):
                self._preload.append(new_preload[idx])

        return self._preload.pop()

    def __next__(self):
        obs, prev_actions, oracle_actions= self._load_next()
        instruction_batch = obs['instruction'][0]
        instruction_batch = np.expand_dims(instruction_batch, axis=0)
        obs['instruction'] = instruction_batch
        for k, v in obs.items():
            obs[k] = torch.from_numpy(v)

        prev_actions = torch.from_numpy(prev_actions)
        
        
        oracle_actions = torch.from_numpy(oracle_actions)

        inflections = torch.cat(
            [
                torch.tensor([1], dtype=torch.long),
                (oracle_actions[1:] != oracle_actions[:-1]).long(),
            ]
        )
        return (obs, prev_actions, oracle_actions, self.inflec_weights[inflections])

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:
            start = 0
            end = self.length
        else:
            per_worker = int(np.ceil(self.length / worker_info.num_workers))

            start = per_worker * worker_info.id
            end = min(start + per_worker, self.length)

        # Reverse so we can use .pop()
        self.load_ordering = list(
            reversed(_block_shuffle(list(range(start, end)), self.preload_size))
        )

        return self

In [ ]:
# lmdb_features_dir = 'data/trajectories_dirs/seq2seq/trajectories.lmdb'
lmdb_features_dir = '/data/zirshad/VLNCE-data/data/trajectory_dir/seq2seq_sem_attn/trajectories.lmdb'

USE_IW= False

dataset = IWTrajectoryDataset(
    lmdb_features_dir,
    USE_IW,
    inflection_weight_coef=1.0,
    lmdb_map_size=1e9,
    batch_size=6,
)
diter = torch.utils.data.DataLoader(
    dataset,
    batch_size=6,
    shuffle=False,
    collate_fn=collate_fn,
    pin_memory=False,
    drop_last=True,  # drop last batch if smaller
    num_workers=0,
)

In [ ]:
print(dataset.length)

In [ ]:
device = (
    torch.device("cuda", config.TORCH_GPU_ID)
    if torch.cuda.is_available()
    else torch.device("cpu")
)
observations_batch,prev_actions_batch,not_done_masks,corrected_actions_batch,weights_batch = next(iter(diter))
# print(observations_batch['depth_features'].shape)
# print(prev_actions_batch.shape)
observations_batch = {
    k: v.to(device=device, non_blocking=True)
    for k, v in observations_batch.items()
}

# rgb_out_dim = observations_batch['rgb'].shape[3]
# rgb_out_dim = observations_batch['depth'].shape[3]

# prev_actions     = prev_actions.view(T,N)
# masks            = masks.view(T,N)
# rgb_embedding    = rgb_embedding.view(self.batch_size, max_len, -1, rgb_out_dim, rgb_out_dim)
# depth_embedding  = depth_embedding.view(self.batch_size, max_len, -1, depth_out_dim, depth_out_dim)

for k, v in observations_batch.items():
    print(v.shape)

In [ ]:
print(observations_batch['progress'].dim())

In [ ]:
print(dataset.length)
print(observations_batch['instruction'].shape)
# print(observations_batch['ego_sem_map'].shape)
print(len(prev_actions_batch))

max_traj_len = max(ele.size(0) for ele in prev_actions_batch)
print(corrected_actions_batch.size())
print(prev_actions_batch.size())

print(corrected_actions_batch.shape)
print(weights_batch.shape)

print(observations_batch['rgb_features'].shape)
print(observations_batch['depth_features'].shape)

T,N = corrected_actions_batch.shape

print(not_done_masks.shape)

# print(prev_actions_batch.view(T,N))

In [ ]:
# print(corrected_actions_batch)
# print(observations_batch['instruction'][1])



print(observations_batch['rgb_features'].shape)
print(observations_batch['depth_features'].shape)

lengths = (observations_batch['instruction'] != 0.0).long().sum(dim=1)

print(prev_actions_batch.view(T,N).shape)

# Setup Actor Critic Agent /Model

In [ ]:
from typing import Dict

def load_checkpoint(checkpoint_path, *args, **kwargs) -> Dict:
    r"""Load checkpoint of specified path as a dict.

    Args:
        checkpoint_path: path of target checkpoint
        *args: additional positional args
        **kwargs: additional keyword args

    Returns:
        dict containing checkpoint info
    """
    return torch.load(checkpoint_path, *args, **kwargs)

ckpt_path = '/data/zirshad/VLNCE-data/data/checkpoints/seq2seq_text_attn/ckpt.24.pth'

# from vlnce_baselines.models.cma_policy import CMAPolicy
# from vlnce_baselines.models.seq2seq_text_attn import Seq2Seq_Lang_Attn
from vlnce_baselines.models.transformer_policy import TransformerPolicy
import torch

load_from_ckpt = False
device = (
    torch.device("cuda", config.TORCH_GPU_ID)
    if torch.cuda.is_available()
    else torch.device("cpu")
)
model_config = config.MODEL

model_config.defrost()
model_config.TORCH_GPU_ID = config.TORCH_GPU_ID
model_config.freeze()
actor_critic = TransformerPolicy(
                observation_space=envs.observation_spaces[0],
                action_space=envs.action_spaces[0],
                model_config=model_config,
                num_processes = config.NUM_PROCESSES,
                batch_size=config.DAGGER.BATCH_SIZE,
            )
actor_critic.to(device)

optimizer = torch.optim.Adam(actor_critic.parameters(), 
                                         lr=config.MODEL.TRANSFORMER.lr, 
                                         betas=(0.9, 0.98), eps=1e-9)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, 
                                                               patience=config.MODEL.TRANSFORMER.scheduler_patience, 
                                                               verbose=True, min_lr=1e-6)

if load_from_ckpt:
    ckpt_dict = load_checkpoint(ckpt_path, map_location="cpu")
    actor_critic.load_state_dict(ckpt_dict["state_dict"])
    
# if config.DAGGER.PRELOAD_LMDB_FEATURES:
#     envs.close()
#     del envs
#     envs = None

# Test Individual Architecture Modules

## Instruction Encoder

In [5]:


import torch.nn as nn
observations_batch = batch
device = (
    torch.device("cuda", config.TORCH_GPU_ID)
    if torch.cuda.is_available()
    else torch.device("cpu")
)

observations_batch = {
    k: v.to(device=device, non_blocking=True)
    for k, v in observations_batch.items()
}

model_config = config.MODEL
from vlnce_baselines.models.encoders.language_encoder import LanguageEncoder
instruction_encoder = LanguageEncoder(model_config.INSTRUCTION_ENCODER, device).to(device)
encoder_output,encoder_hidden = instruction_encoder(observations_batch)

print(encoder_hidden[0].shape)
print(encoder_output.shape)
# print(encoder_output.permute(1,0,2)[27][0])
# print(encoder_output.permute(1,0,2)[22][1])
# print(encoder_hidden[0])

print(encoder_output[-1])

print(encoder_hidden[0])

bidir = model_config.INSTRUCTION_ENCODER.bidirectional

num_directions = 2 if bidir else 1
encoder2decoder = nn.Linear(model_config.INSTRUCTION_ENCODER.hidden_size * num_directions, model_config.INSTRUCTION_ENCODER.hidden_size * num_directions).to(device)
print(encoder_hidden[-1].shape)
h_t = torch.tanh(encoder2decoder(encoder_hidden[0][-1]))
c_t = encoder_hidden[1]
h_t = h_t.unsqueeze(0)

encoder_hidden = (h_t, c_t)


/home/zirshad/.conda/envs/habitat/lib/python3.6/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Embedded shape: torch.Size([1, 200, 768])
hidden torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 21, 256])
tensor([[ 0.0824, -0.0214, -0.1457,  ..., -0.0278,  0.0441, -0.0408],
        [ 0.1544,  0.2265,  0.2395,  ..., -0.2030,  0.0568,  0.1381],
        [-0.1056,  0.2373,  0.2702,  ..., -0.1626, -0.0258, -0.1785],
        ...,
        [-0.1240, -0.1151,  0.2143,  ...,  0.0139,  0.2472,  0.0384],
        [ 0.0012, -0.1161,  0.0758,  ...,  0.1469,  0.1748,  0.0948],
        [-0.1288, -0.0899,  0.0236,  ..., -0.0771,  0.1580, -0.0919]],
       device='cuda:1', grad_fn=<SelectBackward>)
tensor([[[-0.0685,  0.1152,  0.0360, -0.1138,  0.0303,  0.0773, -0.0295,
          -0.0534, -0.1413,  0.0747, -0.0195, -0.1507,  0.1109,  0.0946,
          -0.0670, -0.0131, -0.1124,  0.0644, -0.1639,  0.0289,  0.0507,
           0.0817, -0.1222,  0.0652,  0.0090, -0.0028,  0.0868,  0.0397,
          -0.1426, -0.0007,  0.1578,  0.0430, -0.0605, -0.1463,  0.1091,
          -0.0285,  0.0697, -

In [6]:
print(encoder_hidden[0].shape)
print(h_t.shape)
print(c_t.shape)

print(encoder_output.shape)

torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 21, 256])


## RGB + Depth Encoder

In [7]:
model_config = config.MODEL
def create_mask(batchsize, max_length, length,device):
    """Given the length create a mask given a padded tensor"""
    tensor_mask = torch.zeros(batchsize, max_length, dtype = torch.bool)
    for idx, row in enumerate(tensor_mask):
        row[:length[idx]] = 1
    tensor_mask.unsqueeze_(-1)
    return tensor_mask.to(device)

from vlnce_baselines.models.encoders.resnet_encoders import (
    TorchVisionResNet50,
    VlnResnetDepthEncoder,
)
observation_space=envs.observation_spaces[0]
action_space=envs.action_spaces[0]
depth_encoder = VlnResnetDepthEncoder(
                observation_space,
                output_size=model_config.DEPTH_ENCODER.output_size,
                checkpoint=model_config.DEPTH_ENCODER.ddppo_checkpoint,
                backbone=model_config.DEPTH_ENCODER.backbone,
                spatial_output=True,
            )
rgb_encoder = TorchVisionResNet50(
                observation_space, model_config.RGB_ENCODER.output_size, 
                model_config.RGB_ENCODER.resnet_output_size,
                device, 
                spatial_output=True,
            )

rgb_encoder = rgb_encoder.to(device)
depth_encoder = depth_encoder.to(device)

depth_embedding = depth_encoder(observations_batch)
print(depth_embedding.shape)
rgb_embedding = rgb_encoder(observations_batch)
# x = torch.cat([depth_embedding, rgb_embedding], dim=1)

2020-08-05 21:32:29,394 Overwriting CNN input size of depth: (256, 256)


torch.Size([1, 192, 4, 4])


In [8]:
print(rgb_embedding.shape)

depth_embedding = torch.flatten(depth_embedding, 2)
rgb_embedding = torch.flatten(rgb_embedding, 2)

print(rgb_embedding.shape)
print(depth_embedding.shape)

torch.Size([1, 2112, 4, 4])
torch.Size([1, 2112, 16])
torch.Size([1, 192, 16])


## RGB DEPTH PRE-PROCESSING TO GET EMBEDDING

In [9]:
print(rgb_encoder.output_shape[0])
print(model_config.RGB_ENCODER.output_size)

import numpy as np
rgb_linear = nn.Sequential(
                nn.AdaptiveAvgPool1d(1),
                nn.Flatten(),
                nn.Linear(
                    rgb_encoder.output_shape[0],
                    model_config.RGB_ENCODER.output_size,
                ),
                nn.ReLU(True),
            ).to(device)

depth_linear = nn.Sequential(
                nn.Flatten(),
                nn.Linear(
                    np.prod(depth_encoder.output_shape),
                    model_config.DEPTH_ENCODER.output_size,
                ),
                nn.ReLU(True),
            ).to(device)

rgb_in = rgb_linear(rgb_embedding)
depth_in = depth_linear(depth_embedding)

print(rgb_in.shape)
print(depth_in.shape)

2112
512
torch.Size([1, 512])
torch.Size([1, 128])


## Initial RNN Encoder

In [33]:
prev_actions = torch.zeros(
    config.NUM_PROCESSES, 1, device=device, dtype=torch.long
)


masks = torch.zeros(config.NUM_PROCESSES, 1, device=device)
num_actions=envs.action_spaces[0].n
prev_action_embedding = nn.Embedding(num_actions + 1, 32).to(device)
prev_actions = prev_action_embedding(
    ((prev_actions.float() + 1) * masks).long().view(-1)
)

from habitat_baselines.rl.models.rnn_state_encoder import RNNStateEncoder
from habitat_baselines.rl.models.rnn_state_encoder import RNNStateEncoder
rnn_input_size = model_config.DEPTH_ENCODER.output_size
rnn_input_size += model_config.RGB_ENCODER.output_size
rnn_input_size += prev_action_embedding.embedding_dim


state_encoder = RNNStateEncoder(
    input_size=rnn_input_size,
    hidden_size=model_config.STATE_ENCODER.hidden_size,
    num_layers=1,
    rnn_type=model_config.STATE_ENCODER.rnn_type,
).to(device)

num_recurrent_layers = state_encoder.num_recurrent_layers + (
            state_encoder.num_recurrent_layers
        )

rnn_hidden_states = torch.ones(
    num_recurrent_layers,
    config.NUM_PROCESSES,
    config.MODEL.STATE_ENCODER.hidden_size,
    device=device,
)



print(num_recurrent_layers)



print(prev_actions.shape)






print(state_encoder.num_recurrent_layers)

print("rnn hidden states",rnn_hidden_states.shape)

state_in = torch.cat([rgb_in, depth_in, prev_actions], dim=1)
(
    state,
    rnn_hidden_states[0 : state_encoder.num_recurrent_layers],
) = state_encoder(
    state_in,
    rnn_hidden_states[0 : state_encoder.num_recurrent_layers],
    masks,
)

print(state.shape)

print(rnn_hidden_states.shape)

4
torch.Size([1, 32])
2
rnn hidden states torch.Size([4, 1, 256])
torch.Size([1, 256])
torch.Size([4, 1, 256])


### Text Attention

In [38]:
scale = torch.tensor(1.0 / ((hidden_size // 2) ** 0.5))

print("scale", scale)

import torch.nn.functional as F
def attn(q, k, v, scale, mask=None):
    logits = torch.einsum("nc, nci -> ni", q, k)

    if mask is not None:
        logits = logits - mask.float() * 1e8

    attn = F.softmax(logits * scale, dim=1)

    return torch.einsum("ni, nci -> nc", attn, v)

instruction_embedding = encoder_output.permute(0,2,1)
hidden_size = model_config.STATE_ENCODER.hidden_size

print(hidden_size)

state_q = torch.nn.Linear(hidden_size, hidden_size // 2).to(device)
text_k = torch.nn.Conv1d(
    instruction_encoder.output_size, hidden_size // 2, 1
).to(device)
text_q = nn.Linear(instruction_encoder.output_size, hidden_size // 2)

text_state_q = state_q(state)
text_state_k = text_k(instruction_embedding)
text_mask = (instruction_embedding == 0.0).all(dim=1)
text_embedding = attn(
    text_state_q, text_state_k, instruction_embedding, scale, text_mask
)

print(text_embedding.shape)

scale tensor(0.0884)
256
torch.Size([1, 256])


## Visual Attention

In [40]:
print(rgb_encoder.output_shape[0])

rgb_kv = nn.Conv1d(
    rgb_encoder.output_shape[0],
    hidden_size // 2 + model_config.RGB_ENCODER.output_size,
    1,
).to(device)

a= rgb_kv(rgb_embedding)

print(a.shape)

print(hidden_size // 2)

2112
torch.Size([1, 640, 16])
128


In [41]:
rgb_kv = nn.Conv1d(
    rgb_encoder.output_shape[0],
    hidden_size // 2 + 128,
    1,
).to(device)

depth_kv = nn.Conv1d(
    depth_encoder.output_shape[0],
    hidden_size // 2 + model_config.DEPTH_ENCODER.output_size,
    1,
).to(device)


rgb_k, rgb_v = torch.split(rgb_kv(rgb_embedding), hidden_size // 2, dim=1
)
depth_k, depth_v = torch.split(
    depth_kv(depth_embedding), hidden_size // 2, dim=1
)

print(rgb_k.shape)

torch.Size([1, 128, 16])


In [ ]:
def create_mask(batchsize, max_length, length,device):
    """Given the length create a mask given a padded tensor"""
    tensor_mask = torch.zeros(batchsize, max_length, dtype = torch.bool)
    print(tensor_mask.shape)
    for idx, row in enumerate(tensor_mask):
        row[:length[idx]] = 1
    tensor_mask.unsqueeze_(-1)
    return tensor_mask.to(device)

# not_done_masks = torch.ones(config.NUM_PROCESSES, 1, device=device)

# print(not_done_masks)

# not_done_masks[0] = 0

# def mask_hidden(hidden_states, masks):
#     if isinstance(hidden_states, tuple):
#         hidden_states = tuple(v * masks for v in hidden_states)
#     else:
#         hidden_states = masks * hidden_states

#     return hidden_states

# decoder_hidden = encoder_hidden
# print(decoder_hidden[0])
# print(1-not_done_masks)

# done_masks = 1- not_done_masks
# print(not_done_masks.unsqueeze(0))



# hidden_states = mask_hidden(decoder_hidden, done_masks.unsqueeze(0))

# print(hidden_states[1])



## Decoder

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
prev_actions = torch.zeros(
    config.NUM_PROCESSES, 1, device=device, dtype=torch.long
)
not_done_masks = torch.zeros(config.NUM_PROCESSES, 1, device=device)

not_done_masks[0] = 0

# Init the RNN state decoder

# observations, prev_actions, not_done_masks, corrected_actions, weights= observations_batch, prev_actions_batch.to(device=device, non_blocking=True), not_done_masks.to(device=device, non_blocking=True), corrected_actions_batch.to(device=device, non_blocking=True), weights_batch.to(device=device, non_blocking=True)
# T,N = corrected_actions.size()


from vlnce_baselines.models.decoder.attn_decoder import Attn_Decoder,Attn_DecoderSequence
rnn_input_size = (model_config.DEPTH_ENCODER.output_size
    + model_config.RGB_ENCODER.output_size
)
state_encoder = Attn_DecoderSequence(
    attn_model = 'general',
    input_size=rnn_input_size,
    hidden_size=model_config.STATE_ENCODER.hidden_size,
    num_layers=1,
    rnn_type=model_config.STATE_ENCODER.rnn_type,
).to(device)

#RNN Input Size (Seq Length, Batch Size, Input Size)
#RNN Hidden Size (Num_layers, Batch Size, Hidden Size)

# rnn_hidden_states = torch.ones(
#     state_encoder.num_recurrent_layers,
#     N,
#     config.MODEL.STATE_ENCODER.hidden_size,
#     device=device,
# )

decoder_hidden = encoder_hidden
x = torch.cat([depth_embedding, rgb_embedding], dim=1)



num_layers =1
rnn_type = model_config.STATE_ENCODER.rnn_type
rnn = getattr(nn, rnn_type)(
    input_size=rnn_input_size,
    hidden_size=model_config.STATE_ENCODER.hidden_size,
    num_layers=num_layers,
    batch_first = True,
).to(device)


batch_size = decoder_hidden[0].size(1)

N = int(x.size(0)/batch_size)

# # unflatten
# x = x.view(batch_size, N, -1)

# x =x[:,0,:]
# x= x.unsqueeze(1)
# print(x.shape)

# # embed_mask = create_mask(x.shape[0], x.shape[1], obs_lengths, device)
# # embed_mask = embed_mask.expand_as(x)
# # x = x*embed_mask

# # print(x.shape)

# # packed_seq = nn.utils.rnn.pack_padded_sequence(x, obs_lengths, batch_first=True, enforce_sorted=False)
# output, hidden_states = rnn(x, decoder_hidden)

# attn = nn.Linear(model_config.STATE_ENCODER.hidden_size, model_config.STATE_ENCODER.hidden_size, bias=False).to(device)
# method ='general'
# batch_size, output_len, dimensions = output.size()
# max_len = encoder_output.size(1)

# if method == "general":
#     output = output.reshape(batch_size * output_len, dimensions)
#     output = attn(output)
#     output = output.reshape(batch_size, output_len, dimensions)
    
# print(output.shape)

# print(encoder_output.transpose(1,2).shape)
# attention_scores = torch.bmm(output, encoder_output.transpose(1, 2).contiguous())

# print(attention_scores.shape)

# # print(output.shape)

# # print(attention_scores.shape)

# # print(attention_scores[1,0,:])

# mask = create_mask(encoder_output.shape[0], encoder_output.shape[1], lengths, device)
# mask = mask.permute(0,2,1)
# mask = mask.expand(encoder_output.shape[0], output_len, encoder_output.shape[1])

# # print(mask.shape)

# attention_scores.data.masked_fill_(mask == 0, -float('inf'))

# print(attention_scores)
# # print(attention_scores[1,20,:])

# attention_weights = F.softmax(attention_scores, dim=-1)

# print(attention_weights)
# attention_weights = attention_weights.view(batch_size, output_len, max_len)

# print(attention_weights.shape)
# print(encoder_output.shape)

# context = attention_weights.bmm(encoder_output)

# concat_input = torch.cat((output, context), 2)

# concat_input = concat_input.contiguous().view(batch_size*N, -1)  # flatten

# # print(concat_input.shape)

# # print(context.shape)
# # print(attention_weights.shape)
# # print(output.shape)

# concat = nn.Linear(hidden_size * 2, hidden_size).to(device)
# print(concat_input.shape)

# print(x.size())
# print(decoder_hidden[0].size())
x, rnn_hidden_states, attn_weights = state_encoder(x, decoder_hidden, encoder_hidden, encoder_output, not_done_masks, lengths, device)

# print(x.shape)
# print(attn_weights.shape)

# print(attn_weights.data.shape)

In [ ]:
print(x.shape)

In [ ]:
def mask_hidden(decoder_hidden, encoder_hidden, masks):
    if isinstance(decoder_hidden, tuple):
        print("tupleeee")
        done_masks = 1-masks
        hidden_states = tuple(v * masks + w*done_masks for v,w in zip(decoder_hidden, encoder_hidden))
    else:
        hidden_states = masks * hidden_states
    return hidden_states

# print(hidden_states[0])
# print(encoder_hidden[0])
not_done_masks = torch.ones(config.NUM_PROCESSES, 1, device=device)

not_done_masks[0] = 0
print(not_done_masks)

if not isinstance(hidden_states, tuple):
    dec_hidden = self._unpack_hidden(hidden_states)

# done_masks = 1- not_done_masks
# print(not_done_masks.unsqueeze(0))

# if not isinstance(hidden_states, tuple):
#     hidden_states = self._unpack_hidden(hidden_states)

# enc_hidden = mask_hidden(encoder_hidden, done_masks.unsqueeze(0))
# dec_hidden = mask_hidden(hidden_states, not_done_masks.unsqueeze(0))

# print(enc_hidden.shape)
# print("-----------------")
# print(dec_hidden.shape)

# print("-------------------------")


# hidden_states_final = enc_hidden + dec_hidden

# print(hidden_states_final)


hidden_states = mask_hidden(dec_hidden, encoder_hidden, not_done_masks.unsqueeze(0))

print(dec_hidden)

print("--------------------")

print(encoder_hidden)

print("----------------------")

print(hidden_states)

## Attention

In [ ]:
from vlnce_baselines.models.decoder.attn_decoder import SoftAttn

attn_model = 'general'
hidden_size = model_config.STATE_ENCODER.hidden_size
decoder_max_len = x.shape[1]
# Choose attention model
if attn_model != 'none':
    attn = SoftAttn(attn_model, hidden_size). to(device)
    
attn_weights = attn(x, encoder_output,decoder_max_len,device)

In [ ]:
import torch.nn as nn
print(attn_weights.shape)

print(attn_weights[3,2,:].sum())

concat = nn.Linear(model_config.STATE_ENCODER.hidden_size * 2, model_config.STATE_ENCODER.hidden_size).to(device)

context = attn_weights.bmm(encoder_output) # B x S=1 x N
print(context.shape)
concat_input = torch.cat((x, context), 2)
concat_output = F.tanh(concat(concat_input))
print(concat_output.shape)



In [ ]:
print(x.shape)
print(rnn_hidden_states.shape)
print(encoder_output.shape)
print(encoder_hidden[0][-1].shape)
print(encoder_hidden[1].shape)

print (depth_embedding.shape)
print (rgb_embedding.shape)
print(x.shape)
print(rnn_hidden_states.shape)

print(not_done_masks)


In [ ]:
from vlnce_baselines.common.aux_losses import AuxLosses
recurrent_hidden_states = torch.zeros(
    actor_critic.net.num_recurrent_layers,
    config.NUM_PROCESSES,
    config.MODEL.STATE_ENCODER.hidden_size,
    device=device,
)
prev_actions = torch.zeros(
    config.NUM_PROCESSES, 1, device=device, dtype=torch.long
)
not_done_masks = torch.zeros(config.NUM_PROCESSES, 1, device=device)

optimizer = torch.optim.Adam(
            actor_critic.parameters(), lr=config.DAGGER.LR
        )

for batch in diter:
    (
                            observations_batch,
                            prev_actions_batch,
                            not_done_masks,
                            corrected_actions_batch,
                            weights_batch,
                        ) = batch
    observations_batch = {
                            k: v.to(device=device, non_blocking=True)
                            for k, v in observations_batch.items()
                        }
    observations, prev_actions, not_done_masks, corrected_actions, weights= observations_batch, prev_actions_batch.to(device=device, non_blocking=True), not_done_masks.to(device=device, non_blocking=True), corrected_actions_batch.to(device=device, non_blocking=True), weights_batch.to(device=device, non_blocking=True)
    T, N = corrected_actions.size()
    
    print(T,N)
    optimizer.zero_grad()

    recurrent_hidden_states = torch.zeros(
        actor_critic.net.num_recurrent_layers,
        N,
        config.MODEL.STATE_ENCODER.hidden_size,
        device=device,
    )


    distribution = actor_critic.build_distribution(
        observations, recurrent_hidden_states, prev_actions, not_done_masks
    )

    logits = distribution.logits
#     logits = logits.view(T, N, -1)
    print(logits)
#     print(logits.permute(0, 2, 1).shape)
    corrected_actions = corrected_actions.contiguous().view(T*N)
    print(corrected_actions)
    action_loss = F.cross_entropy(
            logits, corrected_actions, reduction="none"
        )

#     action_loss = F.cross_entropy(
#             logits.permute(0, 2, 1), corrected_actions, reduction="none"
#         )

    action_loss = action_loss.view(T, N)
    
    print(action_loss.shape)
    print(weights_batch.shape)
    print("------------------")
    action_loss = ((weights * action_loss).sum(0) / weights.sum(0)).mean()
    
    

    loss = action_loss 
    loss.backward()
    optimizer.step()
    del distribution, logits, loss, action_loss


In [ ]:
print(recurrent_hidden_states.size(1))

In [ ]:
from vlnce_baselines.common.utils import transform_obs
from habitat_baselines.common.utils import batch_obs

observations = envs.reset()
observations = transform_obs(
    observations, config.TASK_CONFIG.TASK.INSTRUCTION_SENSOR_UUID
)
batch = batch_obs(observations, device=device)




In [ ]:
print(batch['instruction'].shape)

In [ ]:
# def hook_builder(tgt_tensor):
#     def hook(m, i, o):
#         tgt_tensor.set_(o.cpu())
#     return hook
# rgb_features = None
# rgb_hook = None
# if config.MODEL.RGB_ENCODER.cnn_type == "TorchVisionResNet50":
#     rgb_features = torch.zeros((1,), device="cpu")
#     rgb_hook = actor_critic.net.rgb_encoder.layer_extract.register_forward_hook(
#         hook_builder(rgb_features)
#     )
    
# sem_map_attn_features = None
# sem_map_attn_hook = None
# sem_map_attn_features = torch.zeros((1,), device="cpu")
# sem_map_attn_hook = actor_critic.net.sem_map_attn_encoder.ego_sem_model.register_forward_hook(
#         hook_builder(sem_map_attn_features)
#     )

recurrent_hidden_states = torch.zeros(
    actor_critic.net.num_recurrent_layers,
    config.NUM_PROCESSES,
    config.MODEL.STATE_ENCODER.hidden_size,
    device=device,
)
prev_actions = torch.zeros(
    config.NUM_PROCESSES, 1, device=device, dtype=torch.long
)
not_done_masks = torch.zeros(config.NUM_PROCESSES, 1, device=device)
'''
Check if rgb_features is getting updated
'''
(_, actions, _, recurrent_hidden_states) = actor_critic.act(
    batch,
    recurrent_hidden_states,
    prev_actions,
    not_done_masks,
    deterministic=False,
)

print(actions.shape)


In [ ]:
print(actions.view(-1).shape)
print(prev_actions.shape)

In [ ]:
print(batch['lang_attention'].shape)
outputs = envs.step([a[0].item() for a in actions])
observations, _, dones, infos = [list(x) for x in zip(*outputs)]

In [ ]:
print(observations[2]['rgb'].shape)

In [ ]:
observation_space= envs.observation_spaces[0]
if 'ego_sem_map' in observation_space.spaces:
    print('true')
    
sem_is_color = len(
                observation_space.spaces['ego_sem_map'].shape) > 2

print(sem_is_color)

print(batch["ego_sem_map"].shape)

In [ ]:
print(prev_actions_batch.shape)

# Check Model Outputs(Instruction Encoder, State encoder, here)

In [ ]:

from vlnce_baselines.common.aux_losses import AuxLosses
import torch.nn as nn

observations, prev_actions, not_done_masks, corrected_actions, weights= observations_batch, prev_actions_batch.to(device=device, non_blocking=True), not_done_masks.to(device=device, non_blocking=True), corrected_actions_batch.to(device=device, non_blocking=True), weights_batch.to(device=device, non_blocking=True)
T, N = corrected_actions.size()
optimizer.zero_grad()

prev_actions_list =None
AuxLosses.clear()

criterion = nn.CrossEntropyLoss(ignore_index=-1, reduction="none")

distribution = actor_critic.build_distribution(
    observations, prev_actions_list, prev_actions, not_done_masks
)


logits = distribution.logits



corrected_actions = corrected_actions.contiguous().view(T*N)
action_loss = criterion(logits, corrected_actions)

print("action loss",action_loss.shape)
print("without view mean",action_loss.mean())


action_loss = action_loss.view(T, N)
print(((weights * action_loss).sum(0) / weights.sum(0)).shape)
action_loss = ((weights * action_loss).sum(0) / weights.sum(0)).mean()

print("action_loss",action_loss)

# aux_mask = (weights > 0).view(-1)
# aux_loss = AuxLosses.reduce(aux_mask)

loss = action_loss 
# loss.backward()

# optimizer.step()

In [ ]:
print(logits)
print(corrected_actions)

In [ ]:
exp_config ='vlnce_baselines/config/paper_configs/cma.yaml'
config = get_config(exp_config, None)

from habitat_baselines.common.baseline_registry import baseline_registry
trainer_init = baseline_registry.get_trainer(config.TRAINER_NAME)
assert trainer_init is not None, f"{config.TRAINER_NAME} is not supported"
trainer = trainer_init(config)

print(trainer)